In [ ]:
import pandas as pd
docs = pd.read_csv('eval_texts.csv', sep='\t')
docs.head()

In [ ]:
inverted_index = create_inverted_index()

In [ ]:
import pickle
with open('reverse_index.pickle', 'rb') as file:
    obj = pickle.load(file)

In [ ]:
import pickle

with open('reverse_index.pickle', 'wb') as handle:
    pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
from flask import Flask
from flask import request
import nltk.stem.porter as stemmer
import json
import requests
import numpy as np

app = Flask(__name__)

@app.route("/reverseindex", methods=["POST"])
def reverseindex():
    j = request.json
    words = j['data']
    
    documents = list(intersect_all(words, obj))
    
    #if maximal numbers of documents given, then select only the desired amount
    if j.get('max_docs'):
        documents = documents[:j.get('max_docs')]
    
    #get text of found documentss
    documents = docs[docs['id'].apply(lambda x : x in documents)].loc[:, ['id', 'text']].to_dict('records')
    
    return json.dumps({"status":"ok", "got_data":j['data'], "processed_data": documents})


@app.route("/reverseindex/add", methods=['POST'])
def add():
    '''
    Add doc to reverse index
    '''
    pass
    

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=13538)

In [ ]:
import pandas as pd
from collections import Counter

def position_in_text(lst, term):
    '''
    lst - list of tokens
    term - term that you are looking for
    Return all positions of term in the list of tokens(0 = first token) 
    '''
    return [i for i, x in enumerate(lst) if x == term]

def create_inverted_index():
    '''
    Return inverted index in list of dictionaries
    word -> [{documentID, count of occurencies in document, positions in document}, ...]
    '''
    inverted_index = dict()
    docs = pd.read_csv('../../Data/eval_texts.csv', sep='\t')
    docs = docs.drop_duplicates(subset='id', keep='first', inplace=True)
    for i in range(docs.shape[0]):
        doc = dict()
        doc['docID'] = int(docs.loc[i, 'id'])
        #doc['text'] = docs.loc[i, 'text_searchable']
        response_analyze = requests.post('http://127.0.0.1:13533/analyze', json={'data' : docs.loc[i, 'text']})
        doc['text_searchable'] = json.loads(response_analyze.text)['words']
        
        #tokens = doc['text'].split(' ')
        tokens = doc['text_searchable']
        number_of_occurrences = Counter(tokens)         
        for term in set(tokens):
          
          if term not in inverted_index:
            inverted_index[term] = [ {'docID': doc['docID'], 'count': number_of_occurrences[term], 'pos': position_in_text(tokens, term)}]
          
          else:
            inverted_index[term].append({'docID': doc['docID'], 'count': number_of_occurrences[term], 'pos': position_in_text(tokens, term)})
            
    return(inverted_index)

In [ ]:
def intersect_all(terms, inverted_index):
    '''
    terms - list of terms for wich we want to intersect set of documents
    inverted_index - created inverted index
    '''
    ans = set()
    for term in terms:            
        p = inverted_index.get(term)
        posting_list=set()
        if not p:
            return set()
        for d in p:
            posting_list.add(d['docID'])
        if len(ans) == 0:
            ans = posting_list
        else:
            ans = ans & posting_list

    return ans